In [2]:
#Loading Dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from os import listdir
from os.path import isfile, join

In [3]:
#Setting the CSV variables
population_census = "Resources/population_by_zip_2010.csv"
restaurant_data = "Resources/FastFoodRestaurants.csv"

In [15]:
#Loading CSV into DataFrames
populationDF = pd.read_csv(population_census, encoding="utf-8")
restaurantDF = pd.read_csv(restaurant_data, encoding="utf-8")

#Changing DataType to match, INT --> STRING over OBJECT | OBJECT DOES NOT WORK USE STRING 
#NOTE; restaurant DF can not be changed to INT due to some zipcodes having a '-'. 
#NOTE; Because of this '-' we must change the datatype to an OBJECT instead, treat as a text
populationDF['zipcode'] = populationDF['zipcode'].astype(str)

In [22]:
#Cleaning DFs, taking only the columns we need
cleaned_population = populationDF[['zipcode','population']].copy()
cleaned_restaurant = restaurantDF[['city','name','postalCode','province']].copy()

#Renaming columns
cleaned_restaurant.rename(columns={'postalCode':'zipcode'}, inplace=True)

#Setting zipcode as index
#NOTE, setting the index for zip code might NOT be necessary; experiementing
#NOTE, renaming the zipcode column in both DFs to a singular name may be beneficial; experimenting
#FINAL NOTE: SETTING INDEX NOTE NECESSARY; we went with a loc function alternatively to take rows with NAN values in to get final population
merged_df = cleaned_restaurant.merge(populationDF, how='outer', on='zipcode')

#Here we are locating the rows where there is NAN values, due to the census having more information then necessary.
#We want the final TOTAL population; not the population divided by age\gender
cleaned_merged = merged_df.loc[merged_df[['minimum_age', 'maximum_age', 'gender']].isnull().all(axis=1),:]
cleaned_merged

#Here we are doing our final clean of our merged file. Dropping unneccessary columns. 
cleaned_merged_final = cleaned_merged[['city','name','zipcode','province','population']].copy()
cleaned_merged_final

#Dropping rows where there are no restaurants within a zipcode. 
cleaned_merged_noNAN = cleaned_merged_final.dropna(how='any')
cleaned_merged_noNAN

,city,name,zipcode,province,population
0,Massena,McDonald's,13662,NY,16582.0
32,Massena,McDonald's,13662,NY,16582.0
64,Massena,Taco Bell,13662,NY,16582.0
96,Washington Court House,Wendy's,43160,OH,22305.0
128,Washington Court House,McDonald's,43160,OH,22305.0
160,Washington Court House,Arby's,43160,OH,22305.0
192,Washington Court House,McDonald's,43160,OH,22305.0
224,Washington Court House,Rancher's Roast Beef Inc.,43160,OH,22305.0
256,Washington Court House,Taco Bell,43160,OH,22305.0
288,Maysville,Frisch's Big Boy,41056,KY,14228.0


,zipcode,population
0,99776,124
1,38305,49808
2,37086,31513
3,41667,720
4,67001,1676


,city,name,zipcode,province
0,Massena,McDonald's,13662,NY
1,Washington Court House,Wendy's,43160,OH
2,Maysville,Frisch's Big Boy,41056,KY
3,Massena,McDonald's,13662,NY
4,Athens,OMG! Rotisserie,45701,OH
